In [4]:
import sys
sys.path.append(".../RADAR_Plataform/RADAR") # dir library


In [5]:
from RADAR.time_series.algorithms import transformers
from RADAR.time_series.time_series_utils import TimeSeriesProcessor
from RADAR.visualization_module import DataVisualization, DataVisualizationScoresTS
import numpy as np
import torch

2025-10-14 13:58:12.216865: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-14 13:58:12.229436: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760443092.243481   41992 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760443092.247428   41992 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-14 13:58:12.262561: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

load and preprocess data


In [6]:
from RADAR.time_series.time_series_datasets_uci import global_load as load_time_series 
from sklearn.model_selection import train_test_split
X,y = load_time_series('ai4i_2020_predictive_maintenance_dataset')   #name dataset in static datasets uci repo
labels = y["Machine failure"]
X = X.drop('Type',axis=1)  # remove Type or apply one hot encoding

Metadata: {'uci_id': 601, 'name': 'AI4I 2020 Predictive Maintenance Dataset', 'repository_url': 'https://archive.ics.uci.edu/dataset/601/ai4i+2020+predictive+maintenance+dataset', 'data_url': 'https://archive.ics.uci.edu/static/public/601/data.csv', 'abstract': 'The AI4I 2020 Predictive Maintenance Dataset is a synthetic dataset that reflects real predictive maintenance data encountered in industry.', 'area': 'Computer Science', 'tasks': ['Classification', 'Regression', 'Causal-Discovery'], 'characteristics': ['Multivariate', 'Time-Series'], 'num_instances': 10000, 'num_features': 6, 'feature_types': ['Real'], 'demographics': [], 'target_col': ['Machine failure', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF'], 'index_col': ['UID', 'Product ID'], 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2020, 'last_updated': 'Wed Feb 14 2024', 'dataset_doi': '10.24432/C5HS5C', 'creators': [], 'intro_paper': {'ID': 386, 'type': 'NATIVE', 'title': 'Explainable Artificial 

In [7]:
from RADAR.time_series.preprocessing.preprocessing_ts import StandardScalerPreprocessing 
scaler = StandardScalerPreprocessing()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, labels, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape )


(8000, 5) (2000, 5) (8000,) (2000,)


##### Define Model Vanilla Transformer

In [8]:

batch_size = 16
seq_len = 24
input_dim = 5  # tanto encoder como decoder
d_model = 64
train_epochs = 10



# Instanciamos el modelo Transformer
kwargs ={
    "algorithm_": "transformer",
    "label_parser": None,
    "size_enc_in":input_dim,
    "size_dec_in": input_dim,
    "ulayers_feedfwd": 128,
    "seq_len":seq_len,
    "d_qk":64,
    "d_v":64,
    "d_model":d_model,
    "n_layers": 2,
    "n_heads": 8,
   "dropout_rate": 0.1,
   "attns_outs":False, 
    "train_epochs": train_epochs,
    "batch_size": batch_size,
    "lr": 1e-3
    }
    

model = transformers.TransformersAnomalyDetection(**kwargs)

Train Params: {'label_parser': None, 'train_epochs': 10, 'batch_size': 16, 'lr': 0.001} 
Model Params: {'algorithm_': 'transformer', 'size_enc_in': 5, 'size_dec_in': 5, 'ulayers_feedfwd': 128, 'seq_len': 24, 'd_qk': 64, 'd_v': 64, 'd_model': 64, 'n_layers': 2, 'n_heads': 8, 'dropout_rate': 0.1, 'attns_outs': False}


Time Series Dataset

In [9]:
processor = TimeSeriesProcessor(window_size= seq_len, step_size=1, future_prediction=False)
X_train_windows, y_train_windows, X_test_windows, y_test_windows = processor.process_train_test(X_train, y_train, X_test, y_test)
print("X_train shape:", X_train_windows.shape)
print("y_train shape:", y_train_windows.shape)
print("X_test shape:", X_test_windows.shape)
print("y_test shape:", y_test_windows.shape)

X_train shape: (7977, 24, 5)
y_train shape: (7977, 24)
X_test shape: (1977, 24, 5)
y_test shape: (1977, 24)


In [10]:

model.fit(X_train_windows)


/data/Beatriz/Doctorado GR/RADAR_Plataform/RADAR/.venv/lib/python3.9/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([16, 24, 5])) that is different to the input size (torch.Size([24, 5])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/data/Beatriz/Doctorado GR/RADAR_Plataform/RADAR/.venv/lib/python3.9/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([9, 24, 5])) that is different to the input size (torch.Size([24, 5])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/10, Loss: 1.007264
Epoch 2/10, Loss: 0.979617
Epoch 3/10, Loss: 0.975353
Epoch 4/10, Loss: 0.996108
Epoch 5/10, Loss: 1.003654
Epoch 6/10, Loss: 1.002857
Epoch 7/10, Loss: 1.002469
Epoch 8/10, Loss: 1.002379
Epoch 9/10, Loss: 1.002286
Epoch 10/10, Loss: 1.002137


In [11]:
scores = model.decision_function(X_train_windows)
labels = model.predict(X_train_windows)

print("Anomaly scores:", scores.shape)
print("Predicted labels:", labels.shape)


Anomaly scores: torch.Size([7977])
Predicted labels: torch.Size([191448, 5])


In [12]:
model.evaluate(X_test_windows,y_test_windows)

Accuracy: 96.291%
F1 Score: 0.142
Recall: 0.104
Precision: 0.225


{'scores': array([ 0.43089256,  1.0358208 ,  0.6022974 , ...,  1.6290067 ,
        12.2048235 ,  1.5705751 ], dtype=float32),
 'labels_preds': array([0, 0, 0, ..., 0, 1, 0]),
 'labels_true': tensor([0, 1, 0,  ..., 0, 1, 0])}

##### Define Model Informer

In [9]:
batch_size = 16
seq_len = 24
pred_len = 24
label_len = 24
input_dim = 5  # tanto encoder como decoder
d_model = 64
train_epochs = 5

# Simulamos series de entrada

# Instanciamos el modelo Transformer
kwargs = {
    "algorithm_": "informer",
    "label_parser": None,
    "enc_in": input_dim,             # Number of input variables for the encoder
    "dec_in": input_dim,             # Number of input variables for the decoder
    "c_out": input_dim,              # Output dimension (change if needed)
    "seq_len": seq_len,              # Input sequence length
    "label_len": label_len,          # Length of the decoder input (label segment)
    "out_len": pred_len,             # Prediction length
    "factor": 5,                     # ProbSparse factor (used in ProbAttention)
    "d_model": d_model,              # Model dimension
    "n_heads": 8,                    # Number of attention heads
    "e_layers": 2,                   # Number of encoder layers
    "d_layers": 1,                   # Number of decoder layers
    "d_ff": 128,                     # Feedforward network dimension
    "dropout": 0.1,                  # Dropout rate
    "attn": "prob",                  # Attention type: 'prob' or 'full'
    "activation": "gelu",           # Activation function
    "output_attention": False,       # Whether to output attention weights
    "distil": True,                  # Whether to use distillation in the encoder
    "mix": True,                     # Whether to use mixed attention in the decoder
    "train_epochs": train_epochs,    # Number of training epochs
    "batch_size": batch_size,        # Batch size
    "lr": 1e-3                       # Learning rate
}
    

model = transformers.TransformersAnomalyDetection(**kwargs)

Train Params: {'label_parser': None, 'train_epochs': 5, 'batch_size': 16, 'lr': 0.001} 
Model Params: {'algorithm_': 'Informer', 'enc_in': 5, 'dec_in': 5, 'c_out': 5, 'seq_len': 24, 'label_len': 24, 'out_len': 24, 'factor': 5, 'd_model': 64, 'n_heads': 8, 'e_layers': 2, 'd_layers': 1, 'd_ff': 128, 'dropout': 0.1, 'attn': 'prob', 'activation': 'gelu', 'output_attention': False, 'distil': True, 'mix': True}


Time Serie Dataset

In [6]:
processor = TimeSeriesProcessor(window_size= seq_len, step_size=1, future_prediction=False)
X_train_windows, y_train_windows, X_test_windows, y_test_windows = processor.process_train_test(X_train, y_train, X_test, y_test)
print("X_train shape:", X_train_windows.shape)
print("y_train shape:", y_train_windows.shape)
print("X_test shape:", X_test_windows.shape)
print("y_test shape:", y_test_windows.shape)

X_train shape: (7977, 24, 5)
y_train shape: (7977, 24)
X_test shape: (1977, 24, 5)
y_test shape: (1977, 24)


Fit and Evaluate

In [7]:
model.fit(X_train_windows)


/data/Beatriz/Doctorado GR/ADL_platform/S-ADL/venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([16, 24, 5])) that is different to the input size (torch.Size([24, 5])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/data/Beatriz/Doctorado GR/ADL_platform/S-ADL/venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([9, 24, 5])) that is different to the input size (torch.Size([24, 5])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/5, Loss: 1.018134
Epoch 2/5, Loss: 0.987470
Epoch 3/5, Loss: 0.974741
Epoch 4/5, Loss: 0.964567
Epoch 5/5, Loss: 0.959280


In [8]:
scores = model.decision_function(X_test_windows)
pred = model.predict(X_test_windows)

print("Anomaly scores:", scores.shape)
print("Predicted labels:", pred.shape)

Anomaly scores: torch.Size([1977])
Predicted labels: torch.Size([47448, 5])


In [ ]:
model.evaluate(X_test_windows,y_test_windows)

In [20]:
labels_pred = model.labels_preds
labels_true = np.array(y_test).ravel()

In [ ]:
visualizer = DataVisualization(data=X_test, plot_technique='plot_anomaly', dim_reduction_technique='PCA', y_true=labels_true, y_pred=labels_pred,subset_size_percent=0.2)
visualizer.fit()
visualizer.show()

##### Define Model Autoformer

In [5]:
batch_size = 16
seq_len = 24
pred_len = 24
label_len = 24
input_dim = 5       
d_model = 64
train_epochs = 5

kwargs = {
    "algorithm_": "autoformer",
    "label_parser": None,
    "seq_len": seq_len,
    "label_len": label_len,
    "pred_len": pred_len,
    "enc_in": input_dim,
    "dec_in": input_dim,
    "c_out": input_dim,
    "d_model": d_model,
    "n_heads": 8,
    "e_layers": 2,
    "d_layers": 1,
    "d_ff": 128,
    "moving_avg": 25,
    "factor": 5,
    "dropout": 0.1,
    "activation": "gelu",
    "output_attention": False,
    "train_epochs": train_epochs,    # Number of training epochs
    "batch_size": batch_size,        # Batch size
    "lr": 1e-3                       # Learning rate
}

modelAutoformer = transformers.TransformersAnomalyDetection(**kwargs)

Train Params: {'label_parser': None, 'train_epochs': 5, 'batch_size': 16, 'lr': 0.001} 
Model Params: {'algorithm_': 'autoformer', 'seq_len': 24, 'label_len': 24, 'pred_len': 24, 'enc_in': 5, 'dec_in': 5, 'c_out': 5, 'd_model': 64, 'n_heads': 8, 'e_layers': 2, 'd_layers': 1, 'd_ff': 128, 'moving_avg': 25, 'factor': 5, 'dropout': 0.1, 'activation': 'gelu', 'output_attention': False}


Time series Dataset

In [6]:
processor = TimeSeriesProcessor(window_size= seq_len, step_size=1, future_prediction=False)
X_train_windows, y_train_windows, X_test_windows, y_test_windows = processor.process_train_test(X_train, y_train, X_test, y_test)
print("X_train shape:", X_train_windows.shape)
print("y_train shape:", y_train_windows.shape)
print("X_test shape:", X_test_windows.shape)
print("y_test shape:", y_test_windows.shape)

X_train shape: (7977, 24, 5)
y_train shape: (7977, 24)
X_test shape: (1977, 24, 5)
y_test shape: (1977, 24)


Fit and Evaluate

In [7]:
modelAutoformer.fit(X_train_windows)


/data/Beatriz/Doctorado GR/ADL_platform/S-ADL/venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([16, 24, 5])) that is different to the input size (torch.Size([24, 5])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/data/Beatriz/Doctorado GR/ADL_platform/S-ADL/venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([9, 24, 5])) that is different to the input size (torch.Size([24, 5])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/5, Loss: 1.018365
Epoch 2/5, Loss: 0.991262
Epoch 3/5, Loss: 0.985227
Epoch 4/5, Loss: 0.982086
Epoch 5/5, Loss: 0.980459


In [8]:
scores = modelAutoformer.decision_function(X_test_windows)
pred = modelAutoformer.predict(X_test_windows)

print("Anomaly scores:", scores.shape)
print("Predicted labels:", pred.shape)

Anomaly scores: torch.Size([1977])
Predicted labels: torch.Size([47448, 5])


In [9]:
modelAutoformer.evaluate(X_test_windows,y_test_windows)

Accuracy: 96.217%
F1 Score: 0.131
Recall: 0.096
Precision: 0.204


{'scores': array([0.23424728, 0.6546804 , 0.93976355, ..., 2.4287481 , 9.388143  ,
        2.4993958 ], dtype=float32),
 'labels_preds': array([0, 0, 0, ..., 0, 1, 0]),
 'labels_true': tensor([0, 1, 0,  ..., 0, 1, 0])}

In [10]:
labels_pred = modelAutoformer.labels_preds
labels_true = np.array(y_test).ravel()

Only for when there are y_true or anomaly labels

In [11]:
visualizer = DataVisualization(data=X_test, plot_technique='plot_anomaly', dim_reduction_technique='PCA', y_true=labels_true, y_pred=labels_pred,subset_size_percent=0.2)
visualizer.fit()
visualizer.show()

Visualizing anomalies in temporal data from scores

In [12]:

visualizer = DataVisualizationScoresTS(scores)
fig = visualizer.visualize(method="percentile", threshold=0.95)  # percentil
fig.show()

In [ ]:
fig2 = visualizer.visualize(method="std", threshold=3)            # std
fig3 = visualizer.visualize(method="topk", top_k=2)               # top-k
fig2.show()